#Name: Ankita Ghosh
#Reg number: 180905354
#Sec:A Roll number:41

Colab Link: https://colab.research.google.com/drive/1XpQZLwTLfm79ByrLqYrXQkBxtsvfZvkl?usp=sharing

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

##Question 1
#####Write a program in CUDA to count the number of times a given word is repeated in a sentence. (Use Atomic function).

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__global__ void FindCount(char* str,int* d_count,int* len_words,char* word)
{

  int tempIdx=threadIdx.x;
  int idx=0;
  for(int i=0;i<tempIdx;i++)
  {
      idx += len_words[i]+1;
  }

  char s[10];
  int x=0;

  for(int j=0;j<len_words[tempIdx];j++){
      s[x++]=str[idx+j];
  }
  s[x] = '\0';

  int flag=0;
  for(int k=0;k<len_words[tempIdx];k++){
      if(s[k]!=word[k]){
          flag=1;
          break;
      }
  }
  if(flag==0){
      atomicAdd(d_count,1);
  }
}

int main(){
  char str[]="My name is Ankita Ghosh and name of the lab is PP";
  printf("Sentence: %s\n\n",str);
  char word[]="name";
  int words_num=1;
  int count=0;
  int final_ans;
  int i=0;
  while(str[i]!='\0')
  {
      if(str[i]==' ')
      words_num++;
      i++;
  }
  int len_words[words_num];
  int len=0,j=0;
  i=0;
  while(str[i] != '\0')
  {
      len++;
      if(str[i]==' '){
          len--;
          len_words[j++]=len;
          len=0;
      }
      i++;
  }
  len_words[j]=len;

  int words_total=sizeof(len_words)/sizeof(int);

 char* d_str;
 int* d_count;
 char* d_word;
 int* d_lenwords;

  cudaMalloc((void**)&d_str,strlen(str)*sizeof(char));
  cudaMalloc((void**)&d_count,sizeof(int));
  cudaMalloc((void**)&d_word,strlen(word)*sizeof(char));
  cudaMalloc((void**)&d_lenwords,words_total*sizeof(int));

  cudaMemcpy(d_str,str,strlen(str)*sizeof(char),cudaMemcpyHostToDevice);
  cudaMemcpy(d_count,&count,sizeof(int),cudaMemcpyHostToDevice);
  cudaMemcpy(d_word,word,strlen(word)*sizeof(char),cudaMemcpyHostToDevice);
  cudaMemcpy(d_lenwords,len_words,words_total*sizeof(int),cudaMemcpyHostToDevice);

  FindCount<<<1,words_total>>>(d_str,d_count,d_lenwords,d_word);

  cudaMemcpy(&final_ans,d_count,sizeof(int),cudaMemcpyDeviceToHost);

  printf("Word: %s\nFrequency in sentence: %d\n",word,final_ans);

  cudaFree(d_str);
  cudaFree(d_count);
  cudaFree(d_word);
  cudaFree(d_lenwords);


}

Sentence: My name is Ankita Ghosh and name of the lab is PP

Word: name
Frequency in sentence: 2



##Question 2
#####Write a CUDA program that reads a string Sin and produces the string Sout as follows:
#####Input string Sin: PCAP              
#####Output string Sout: PCAPPCAPCP

In [ ]:
%%cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__global__ void getNewString(char *Sin,char *Sout,int len)
{
    int tempIdx = threadIdx.x;
    int idx =0;
    for(int i=0;i<tempIdx;i++)
        idx += len-(i);
 
    for(int i=0;i<(len-tempIdx);i++)
        Sout[idx+i] =Sin[i];
}


int main(){
    char Sin[]="PCAP";
    printf("Input String: %s\n",Sin);
    int Sin_len = strlen(Sin);
    int Sout_len = ((Sin_len*(Sin_len+1))/2);
    char Sout[Sout_len];
    char result[Sout_len];

    char *d_Sin;
    char *d_Sout;

    cudaMalloc((void**)&d_Sin,Sin_len*sizeof(char));
    cudaMalloc((void**)&d_Sout,Sout_len*sizeof(char));

    cudaMemcpy(d_Sin,Sin,Sin_len*sizeof(char),cudaMemcpyHostToDevice);
    cudaMemcpy(d_Sout,Sout,Sout_len*sizeof(char),cudaMemcpyHostToDevice);

    getNewString<<<1,Sin_len>>>(d_Sin,d_Sout,Sin_len);

    cudaMemcpy(result,d_Sout,Sout_len*sizeof(char),cudaMemcpyDeviceToHost);
    result[Sout_len] = '\0';

    printf("Output String: %s\n",result);

    cudaFree(d_Sin);
    cudaFree(d_Sout);
}

Input String: PCAP
Output String: PCAPPCAPCP

